In [245]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing import sequence, text
from keras.layers import Input, Embedding
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
import string
from sentence_transformers import SentenceTransformer, LoggingHandler
import numpy as np
import logging

from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
import re
from fastai.tabular import add_datepart

[nltk_data] Downloading package stopwords to /home/ryan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ryan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ryan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
train = pd.read_csv('../MachineHackE-Commerce/Train.csv')
test = pd.read_csv('../MachineHackE-Commerce/Test.csv')

In [3]:
train.head()

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date,Selling_Price
0,P-2610,B-659,bags wallets belts,bags,hand bags,4.3,2/3/2017,291.0
1,P-2453,B-3078,clothing,women s clothing,western wear,3.1,7/1/2015,897.0
2,P-6802,B-1810,home decor festive needs,showpieces,ethnic,3.5,1/12/2019,792.0
3,P-4452,B-3078,beauty and personal care,eye care,h2o plus eye care,4.0,12/12/2014,837.0
4,P-8454,B-3078,clothing,men s clothing,t shirts,4.3,12/12/2013,470.0


## sentence embedding

In [4]:
np.set_printoptions(threshold=100)

logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
### /print debug information to stdout
mo = SentenceTransformer('bert-base-nli-mean-tokens')

2020-06-13 20:08:38 - Load pretrained SentenceTransformer: bert-base-nli-mean-tokens
2020-06-13 20:08:38 - Did not find a '/' or '\' in the name. Assume to download model from server.
2020-06-13 20:08:38 - Load SentenceTransformer from folder: /home/ryan/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_bert-base-nli-mean-tokens.zip
2020-06-13 20:08:38 - loading configuration file /home/ryan/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_bert-base-nli-mean-tokens.zip/0_BERT/config.json
2020-06-13 20:08:38 - Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type

In [5]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

2020-06-13 20:08:44 - Load pretrained SentenceTransformer: bert-base-nli-mean-tokens
2020-06-13 20:08:44 - Did not find a '/' or '\' in the name. Assume to download model from server.
2020-06-13 20:08:44 - Load SentenceTransformer from folder: /home/ryan/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_bert-base-nli-mean-tokens.zip
2020-06-13 20:08:44 - loading configuration file /home/ryan/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_bert-base-nli-mean-tokens.zip/0_BERT/config.json
2020-06-13 20:08:44 - Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type

In [6]:
train['text'] = train.apply(lambda row: ' '.join ([str(row['Item_Category']), str(row['Subcategory_1']),str(row['Subcategory_2'])]), 
                                          axis=1)
test['text'] = test.apply(lambda row: ' '.join ([str(row['Item_Category']), str(row['Subcategory_1']),str(row['Subcategory_2'])]), 
                                          axis=1)



In [7]:
index_train=range(len(train))
index_test=range(len(test))

columns=[i for i in range(0,768)]

emedding_train = pd.DataFrame(index=index_train, columns=columns)
emedding_test = pd.DataFrame(index=index_test, columns=columns)

In [8]:
sentences_train = list(train['text'].values)
sentences_test = list(test['text'].values)

In [9]:
sentence_embeddings_train = model.encode(sentences_train)

Batches: 100%|██████████| 307/307 [00:35<00:00,  8.54it/s]


In [10]:
sentence_embeddings_test = model.encode(sentences_test)

Batches: 100%|██████████| 132/132 [00:14<00:00,  9.18it/s]


In [11]:
for i, embedding in zip(range(len(train)), sentence_embeddings_train):
    emedding_train.loc[i] = embedding
    
for i, embedding in zip(range(len(test)), sentence_embeddings_test):
    emedding_test.loc[i] = embedding    

In [12]:
emedding_train.to_csv('train_embedding.csv' , index=False)
emedding_test.to_csv('test_embedding.csv' , index=False)

In [15]:
emedding_train.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.379569,0.279233,0.277242,0.182598,-0.0770812,-0.026147,-0.752012,0.213204,-0.542107,-0.254378,...,-0.7895,-0.814906,1.40297,-0.431546,-0.808239,-0.416197,-0.109691,-0.240367,-0.860784,-0.724562
1,0.916285,-0.43429,-0.17294,0.477194,0.879464,-0.241531,0.457079,-0.051465,0.554452,0.146344,...,-0.37022,-1.56034,-0.148083,0.256251,0.312532,-1.11397,0.21864,-0.74836,0.103087,-0.56842
2,0.269764,0.285432,1.90438,0.109413,0.202357,0.0958268,0.0449975,-0.75027,1.12793,-0.646643,...,-0.573101,-1.24161,0.136805,-0.388171,-0.704002,-0.604601,-0.340508,-0.507581,-1.18811,-0.299556
3,0.321984,0.791935,0.835692,0.228354,0.771749,0.304709,0.189293,0.361042,0.330234,-0.0185263,...,-0.405189,-1.19135,-0.21445,-0.117667,0.0435669,-0.709908,0.0637145,-0.430006,-0.397084,-0.351611
4,1.534,-0.606129,0.335989,0.388715,0.502638,0.0399542,0.0932059,0.0185919,-0.0460238,-0.539201,...,0.536556,-0.877888,0.963644,0.127429,-0.339267,-0.226999,0.0796143,0.651112,0.199736,-0.291098


## topic modelling
top 15 topics int the data

In [4]:
train = pd.read_csv('../MachineHackE-Commerce/Train.csv')
test = pd.read_csv('../MachineHackE-Commerce/Test.csv')

In [5]:
train['text'] = train.apply(lambda row: ' '.join ([str(row['Item_Category']), str(row['Subcategory_1']),str(row['Subcategory_2'])]), 
                                          axis=1)
test['text'] = test.apply(lambda row: ' '.join ([str(row['Item_Category']), str(row['Subcategory_1']),str(row['Subcategory_2'])]), 
                                          axis=1)



In [6]:
# create count vectorizer first
cvectorizer = CountVectorizer(min_df=4, max_features=4000000, ngram_range=(1,3))

cvz_train = cvectorizer.fit_transform(train['text'])

cvz_test = cvectorizer.transform(test['text'])



In [21]:
# generate topic models using Latent Dirichlet Allocation
lda_model = LatentDirichletAllocation(n_components=15, learning_method='online', max_iter=30, random_state=42)

train_topics = lda_model.fit_transform(cvz_train)
test_topics = lda_model.transform(cvz_test)

In [22]:
n_top_words = 8
topic_summaries = []

# get topics and topic terms
topic_word = lda_model.components_ 
vocab = cvectorizer.get_feature_names()

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))
    print('Topic {}: {}'.format(i, ' | '.join(topic_words)))

Topic 0: jewellery | unknown | rings | jewellery rings | jewellery rings unknown | rings unknown | laptop | accessories
Topic 1: supplies | school | school supplies | toys | computers | toys school supplies | toys school | components
Topic 2: home | furnishing | home furnishing | covers | curtains | linen | bath | towels
Topic 3: clothing | men | men clothing | clothing men | clothing men clothing | clothing shirts | men clothing shirts | shirts
Topic 4: care | and | personal care | personal | beauty and | beauty and personal | and personal | and personal care
Topic 5: clothing | kids | kids clothing | clothing kids clothing | clothing kids | wear | girls | clothing girls
Topic 6: watches | wrist | wrist watches | watches wrist | watches wrist watches | furniture | furnishing | sports
Topic 7: footwear | women | footwear women | women footwear | footwear women footwear | shoes | unknown | casual
Topic 8: bangles | tools | bracelets | jewellery | bangles bracelets armlets | jewellery ba

In [23]:
train_topics_df = pd.DataFrame(train_topics) 
test_topics_df = pd.DataFrame(test_topics) 

In [25]:
train_topics_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.004167,0.004167,0.004167,0.004167,0.004167,0.004167,0.004167,0.004167,0.004167,0.941667,0.004167,0.004167,0.004167,0.004167,0.004167
1,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.928205,0.005128,0.005128,0.005128,0.005128
2,0.004167,0.004167,0.004167,0.004167,0.004167,0.004167,0.004167,0.004167,0.004167,0.004167,0.004167,0.004167,0.004167,0.004167,0.941667
3,0.004762,0.004762,0.147619,0.004762,0.790476,0.004762,0.004762,0.004762,0.004762,0.004762,0.004762,0.004762,0.004762,0.004762,0.004762
4,0.006667,0.006667,0.006667,0.906667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447,0.005128,0.005128,0.005128,0.005128,0.005128,0.928205,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128
2448,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.928205,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128
2449,0.006667,0.006667,0.006667,0.906667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667
2450,0.003509,0.003509,0.003509,0.003509,0.003509,0.003509,0.003509,0.003509,0.003509,0.003509,0.003509,0.003509,0.950877,0.003509,0.003509


In [26]:
train_topics_df = train_topics_df.add_suffix('_topic')
test_topics_df = test_topics_df.add_suffix('_topic')

In [27]:
train_topics_df.to_csv('train_topic.csv', index=False)
test_topics_df.to_csv('test_topic.csv', index=False)

## tf-idf 
but only top 100 words

In [246]:
import numpy as np
import scipy.sparse

In [247]:
train = pd.read_csv('../MachineHackE-Commerce/Train.csv')
test = pd.read_csv('../MachineHackE-Commerce/Test.csv')

In [248]:
train['text'] = train.apply(lambda row: ' '.join ([str(row['Item_Category']), str(row['Subcategory_1']),str(row['Subcategory_2'])]), 
                                          axis=1)
test['text'] = test.apply(lambda row: ' '.join ([str(row['Item_Category']), str(row['Subcategory_1']),str(row['Subcategory_2'])]), 
                                          axis=1)

In [249]:
import re
def tokenizer(text):
    if text:
        result = re.findall('[a-z]{2,}', text.lower())
    else:
        result = []
    return result

In [250]:
# word level tf-idf for model Infoarticle text
vect_word = TfidfVectorizer(max_features=100, tokenizer=tokenizer,analyzer='word', stop_words='english', ngram_range=(1,3), dtype=np.float32) 
vect_word.fit(train['text'].values)
tfidf_train = vect_word.transform(train['text'].values)
tfidf_test = vect_word.transform(test['text'].values)


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [251]:
# tfidf = dict(zip(vect_word.get_feature_names(), vect_word.idf_))
# tfidf = pd.DataFrame(columns=['des_word_tfidf']).from_dict(dict(tfidf), orient='index')
# tfidf.columns = ['des_word_tfidf']


In [252]:
# tfidf.sort_values(by = 'des_word_tfidf' , ascending = False )

In [253]:
# tfidf_train.toarray()

In [254]:
tfidf_test.shape

(1051, 100)

In [261]:
train_tfidf = pd.DataFrame(tfidf_train.toarray()) 
test_tfidf = pd.DataFrame(tfidf_test.toarray()) 

In [262]:
train_tfidf

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.254350,0.313629,0.313629,0.235802,0.257372,0.0,0.313629,0.000000,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.287997,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.386894,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
2448,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.210195,0.000000,0.0,0.000000,0.363598,0.0,0.0
2449,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
2450,0.178583,0.229974,0.229974,0.224758,0.229974,0.229974,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0


In [263]:
train_tfidf = train_tfidf.add_suffix('_tfidf')
test_tfidf = test_tfidf.add_suffix('_tfidf')

In [264]:
train_tfidf.to_csv('train_tfidf.csv', index=False)
test_tfidf.to_csv('test_tfidf.csv', index=False)

In [265]:
train_tfidf

,0_tfidf,1_tfidf,2_tfidf,3_tfidf,4_tfidf,5_tfidf,6_tfidf,7_tfidf,8_tfidf,9_tfidf,...,90_tfidf,91_tfidf,92_tfidf,93_tfidf,94_tfidf,95_tfidf,96_tfidf,97_tfidf,98_tfidf,99_tfidf
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.254350,0.313629,0.313629,0.235802,0.257372,0.0,0.313629,0.000000,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.287997,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.386894,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
2448,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.210195,0.000000,0.0,0.000000,0.363598,0.0,0.0
2449,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
2450,0.178583,0.229974,0.229974,0.224758,0.229974,0.229974,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0


## keras 
dont use it nikka

In [193]:
train = pd.read_csv('../MachineHackE-Commerce/Train.csv')
test = pd.read_csv('../MachineHackE-Commerce/Test.csv')

In [194]:
train['text'] = train.apply(lambda row: ' '.join ([str(row['Item_Category']), str(row['Subcategory_1']),str(row['Subcategory_2'])]), 
                                          axis=1)
test['text'] = test.apply(lambda row: ' '.join ([str(row['Item_Category']), str(row['Subcategory_1']),str(row['Subcategory_2'])]), 
                                          axis=1)

In [195]:
from keras.preprocessing.text import Tokenizer
raw_text = np.hstack([train.Item_Category.str.lower(), train.Subcategory_1.str.lower(), train.Subcategory_2.str.lower()])

tok_raw = Tokenizer()
tok_raw.fit_on_texts(raw_text)

train["seq_item_category"] = tok_raw.texts_to_sequences(train.Item_Category.str.lower())
test["seq_item_category"] = tok_raw.texts_to_sequences(test.Item_Category.str.lower())

train["seq_Subcategory_1"] = tok_raw.texts_to_sequences(train.Subcategory_1.str.lower())
test["seq_Subcategory_1"] = tok_raw.texts_to_sequences(test.Subcategory_1.str.lower())

train["seq_Subcategory_2"] = tok_raw.texts_to_sequences(train.Subcategory_2.str.lower())
test["seq_Subcategory_2"] = tok_raw.texts_to_sequences(test.Subcategory_2.str.lower())


In [197]:
train.head()

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date,Selling_Price,text,seq_item_category,seq_Subcategory_1,seq_Subcategory_2
0,P-2610,B-659,bags wallets belts,bags,hand bags,4.3,2/3/2017,291.0,bags wallets belts bags hand bags,"[47, 67, 74]",[47],"[109, 47]"
1,P-2453,B-3078,clothing,women s clothing,western wear,3.1,7/1/2015,897.0,clothing women s clothing western wear,[1],"[3, 2, 1]","[10, 4]"
2,P-6802,B-1810,home decor festive needs,showpieces,ethnic,3.5,1/12/2019,792.0,home decor festive needs showpieces ethnic,"[12, 20, 36, 41]",[53],[48]
3,P-4452,B-3078,beauty and personal care,eye care,h2o plus eye care,4.0,12/12/2014,837.0,beauty and personal care eye care h2o plus eye...,"[43, 24, 42, 16]","[194, 16]","[573, 334, 194, 16]"
4,P-8454,B-3078,clothing,men s clothing,t shirts,4.3,12/12/2013,470.0,clothing men s clothing t shirts,[1],"[11, 2, 1]","[55, 40]"


## data prep

In [214]:
train = pd.read_csv('../MachineHackE-Commerce/Train.csv')
test = pd.read_csv('../MachineHackE-Commerce/Test.csv')

In [215]:
train.columns

Index(['Product', 'Product_Brand', 'Item_Category', 'Subcategory_1',
       'Subcategory_2', 'Item_Rating', 'Date', 'Selling_Price'],
      dtype='object')

In [216]:
# label encoding for all categorical variables

In [217]:
class LabelEncoderExt(object):
    def __init__(self):
        """
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        from sklearn.preprocessing import LabelEncoder

        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)

In [218]:
def robustlabelencoder(train,test,cols=[]):
    from sklearn.preprocessing import LabelEncoder
    traind = train.copy()
    testd = test.copy()
    label_enc = LabelEncoderExt()
    
    print(cols)
    for col in cols:
        label_enc.fit(train[col])
        traind[col] = label_enc.transform(traind[col])
        testd[col] = label_enc.transform(testd[col])
        
    print(traind.shape,testd.shape)
    
    return traind,testd

In [219]:
train_enc,test_enc = robustlabelencoder(train,test,cols=['Product', 'Product_Brand', 'Item_Category', 'Subcategory_1',
       'Subcategory_2'])

['Product', 'Product_Brand', 'Item_Category', 'Subcategory_1', 'Subcategory_2']
(2452, 8) (1051, 7)


In [224]:
train_enc

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date,Selling_Price
0,791,862,8,11,138,4.3,2/3/2017,291.0
1,760,670,11,128,330,3.1,7/1/2015,897.0
2,1746,280,30,113,102,3.5,1/12/2019,792.0
3,1214,670,9,38,135,4.0,12/12/2014,837.0
4,2104,670,11,81,297,4.3,12/12/2013,470.0
...,...,...,...,...,...,...,...,...
2447,2181,430,11,63,132,2.3,11/3/2017,741.0
2448,130,670,25,129,61,1.9,4/12/2016,1590.0
2449,691,813,11,81,297,1.9,19/4/2019,995.0
2450,1835,124,5,3,58,2.7,1/12/2014,1598.0


In [221]:
train

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date,Selling_Price
0,P-2610,B-659,bags wallets belts,bags,hand bags,4.3,2/3/2017,291.0
1,P-2453,B-3078,clothing,women s clothing,western wear,3.1,7/1/2015,897.0
2,P-6802,B-1810,home decor festive needs,showpieces,ethnic,3.5,1/12/2019,792.0
3,P-4452,B-3078,beauty and personal care,eye care,h2o plus eye care,4.0,12/12/2014,837.0
4,P-8454,B-3078,clothing,men s clothing,t shirts,4.3,12/12/2013,470.0
...,...,...,...,...,...,...,...,...
2447,P-8870,B-2292,clothing,kids clothing,girls wear,2.3,11/3/2017,741.0
2448,P-10672,B-3078,footwear,women s footwear,casual shoes,1.9,4/12/2016,1590.0
2449,P-2134,B-479,clothing,men s clothing,t shirts,1.9,19/4/2019,995.0
2450,P-724,B-133,automotive,accessories spare parts,car interior exterior,2.7,1/12/2014,1598.0


In [226]:
train_enc['Date'] = pd.to_datetime(train_enc['Date'] ,errors='coerce')
test_enc['Date'] = pd.to_datetime(test_enc['Date'] ,errors='coerce')

In [228]:
import holidays
time = pd.DataFrame()

In [229]:
in_graceholiday=[]
for holiday in holidays.IND(years=[2013,2014,2015,2016,2017,2018,2019]).items():
    in_graceholiday.append(holiday[0])

In [230]:
in_graceholiday

[datetime.date(2016, 1, 14),
 datetime.date(2016, 1, 26),
 datetime.date(2016, 8, 15),
 datetime.date(2016, 10, 2),
 datetime.date(2016, 5, 1),
 datetime.date(2016, 12, 25),
 datetime.date(2017, 1, 14),
 datetime.date(2017, 1, 26),
 datetime.date(2017, 8, 15),
 datetime.date(2017, 10, 2),
 datetime.date(2017, 5, 1),
 datetime.date(2017, 12, 25),
 datetime.date(2018, 1, 14),
 datetime.date(2018, 1, 26),
 datetime.date(2018, 8, 15),
 datetime.date(2018, 10, 2),
 datetime.date(2018, 5, 1),
 datetime.date(2018, 12, 25),
 datetime.date(2019, 1, 14),
 datetime.date(2019, 1, 26),
 datetime.date(2019, 8, 15),
 datetime.date(2019, 10, 2),
 datetime.date(2019, 5, 1),
 datetime.date(2019, 12, 25),
 datetime.date(2013, 1, 14),
 datetime.date(2013, 1, 26),
 datetime.date(2013, 8, 15),
 datetime.date(2013, 10, 2),
 datetime.date(2013, 5, 1),
 datetime.date(2013, 12, 25),
 datetime.date(2014, 1, 14),
 datetime.date(2014, 1, 26),
 datetime.date(2014, 8, 15),
 datetime.date(2014, 10, 2),
 datetime.date

In [236]:
graceondate=[]
for i in in_graceholiday:
    twodayago = i - timedelta(days = 4)
    twodayafter = i + timedelta(days = 2)
    for date in pd.date_range(twodayago,twodayafter):
        graceondate.append((str(date).split()[0]))

In [237]:
in_graceperiod = list(set(graceondate))

In [238]:
train_enc['Is_Holiday']= [ 1 if str(val).split()[0] in in_graceperiod else 0 for val in train_enc['Date']]

test_enc['Is_Holiday']= [ 1 if str(val).split()[0] in in_graceperiod else 0 for val in test_enc['Date']]

In [239]:
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    
#     for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
#             'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
    
    for n in ('Year', 'Month', 'Week', 'Day','Dayofweek',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
   # df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [240]:
add_datepart(train_enc, 'Date',drop = True)

In [241]:
add_datepart(test_enc, 'Date',drop = True)

In [243]:
train_enc
test_enc

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Is_Holiday,Year,Month,Week,Day,Dayofweek,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start
0,2452,645,14,88,257,4.3,1,2018,1,2,12,4,False,False,False,False
1,2452,1010,35,12,46,3.0,0,2012,12,51,20,3,False,False,False,False
2,2452,670,11,128,330,1.5,1,2014,1,2,12,6,False,False,False,False
3,2452,155,35,87,210,3.9,1,2019,1,2,12,5,False,False,False,False
4,2452,670,11,128,330,1.4,1,2017,1,2,12,3,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,2452,1010,61,93,52,2.0,0,2014,2,9,28,4,True,False,False,False
1047,2452,710,35,87,210,4.7,1,2019,1,2,12,5,False,False,False,False
1048,2452,124,5,3,58,3.5,1,2014,1,2,12,6,False,False,False,False
1049,2452,1010,38,73,49,2.4,0,2013,6,25,23,6,False,False,False,False


In [244]:
train_enc.to_csv('train_enc.csv', index=False)
test_enc.to_csv('test_enc.csv', index=False)